In [ ]:
import pandas as pd
import numpy as np

In [ ]:
breast_cancer_icd_df = pd.read_csv('/data/volume02/yuba/HC_STRUC/bc_full_icd.csv')
breast_cancer_snomed_df = pd.read_csv('/data/volume02/yuba/HC_STRUC/bc_full_snomed.csv')

prostate_cancer_icd_df = pd.read_csv('/data/volume02/yuba/HC_STRUC/prostate_full_icd.csv')
prostate_cancer_snomed_df = pd.read_csv('/data/volume02/yuba/HC_STRUC/prostate_full_snomed.csv')

kidney_cancer_icd_df = pd.read_csv('/data/volume02/yuba/HC_STRUC/kidney_full_icd.csv')
kidney_cancer_snomed_df = pd.read_csv('/data/volume02/yuba/HC_STRUC/kidney_full_snomed.csv')

paraganglioma_df = pd.read_csv('/data/volume02/yuba/HC_STRUC/paraganglioma_full_snomed.csv')

polyp_icd_df = pd.read_csv('/data/volume02/yuba/HC_STRUC/polyp_full_icd.csv')
polyp_snomed_df = pd.read_csv('/data/volume02/yuba/HC_STRUC/polyp_full_snomed.csv')

thyroid_icd_df = pd.read_csv('/data/volume02/yuba/HC_STRUC/thyroid_full_icd.csv')
thyroid_snomed_df = pd.read_csv('/data/volume02/yuba/HC_STRUC/thyroid_full_snomed.csv')

carney_df = pd.read_csv('/data/volume02/yuba/HC_STRUC/carney_full_icd.csv')

diffuse_icd_df = pd.read_csv('/data/volume02/yuba/HC_STRUC/diffuse_gastric_full_icd.csv')
diffuse_snomed_df = pd.read_csv('/data/volume02/yuba/HC_STRUC/diffuse_gastric_full_snomed.csv')


intestinal_icd_df = pd.read_csv('/data/volume02/yuba/HC_STRUC/intestinal_polyp_full_icd.csv')
intestinal_snomed_df = pd.read_csv('/data/volume02/yuba/HC_STRUC/intestinal_polyp_full_snomed.csv')


melanoma_icd_df = pd.read_csv('/data/volume02/yuba/HC_STRUC/melanoma_full_icd.csv')
melanoma_snomed_df = pd.read_csv('/data/volume02/yuba/HC_STRUC/melanoma_full_snomed.csv')


pc_icd_df = pd.read_csv('/data/volume02/yuba/HC_STRUC/pancreatic_full_icd.csv')
pc_snomed_df = pd.read_csv('/data/volume02/yuba/HC_STRUC/pancreatic_full_snomed.csv')

astrocytoma_icd_df = pd.read_csv('/data/volume02/yuba/HC_STRUC/astrocytoma_full_icd.csv')
astrocytoma_snomed_df = pd.read_csv('/data/volume02/yuba/HC_STRUC/astrocytoma_full_snomed.csv')

#################################################


crc_icd_df = pd.read_csv('/data/volume02/yuba/HC_STRUC/crc_full_icd.csv')
crc_snomed_df = pd.read_csv('/data/volume02/yuba/HC_STRUC/crc_full_snomed.csv')

ec_icd_df = pd.read_csv('/data/volume02/yuba/HC_STRUC/ec_full_icd.csv')
ec_snomed_df = pd.read_csv('/data/volume02/yuba/HC_STRUC/ec_full_snomed.csv')

ov_icd_df = pd.read_csv('/data/volume02/yuba/HC_STRUC/ov_full_icd.csv')
ov_snomed_df = pd.read_csv('/data/volume02/yuba/HC_STRUC/ov_full_snomed.csv')

rc_icd_df = pd.read_csv('/data/volume02/yuba/HC_STRUC/rc_full_icd.csv')
rc_snomed_df = pd.read_csv('/data/volume02/yuba/HC_STRUC/rc_full_snomed.csv')

In [ ]:
ov_snomed_df.dtypes

**DATABASE CONSIST OF 2 DIFFERENT DIAGNOSIS TABLES**
- Required to pass through function before merging for full diagnosis dataframe

In [ ]:
def unique_patients(icd_df, snomed_df, id_col = 'PATIENT_IDENTIFIER', enc_col = 'ENCOUNTER_KEY', int_cols=None):

    icd_df_unique = icd_df.drop_duplicates(subset=[id_col, enc_col], keep='first').copy()
    snomed_df_unique = snomed_df.drop_duplicates(subset=[id_col, enc_col], keep='first').copy()
    
    #overlap logic
    overlap = pd.merge(
        icd_df_unique[[id_col, enc_col]],
        snomed_df_unique[[id_col, enc_col]],
        on=[id_col, enc_col],
        how='inner'
    )
    
    n_patients = overlap[id_col].nunique()
    print(f"{n_patients} unique patient(s) has overlapping encounters and will be dropped.")
    
    if not overlap.empty:
        snomed_df_unique = snomed_df_unique.merge(overlap, on=[id_col, enc_col], how='left', indicator=True)
        snomed_df_unique = snomed_df_unique[snomed_df_unique['_merge'] == 'left_only']
        snomed_df_unique = snomed_df_unique.drop(columns=['_merge'])
        
    merged_df = pd.concat([icd_df_unique, snomed_df_unique], ignore_index=True)
    
    if int_cols:
        for col in int_cols:
            if col in merged_df.columns:
                merged_df[col] = merged_df[col].astype('Int64')
    
    return merged_df
        
    #return icd_df_unique.reset_index(drop=True), snomed_df_unique.reset_index(drop=True)

In [ ]:
bc_df = unique_patients(breast_cancer_icd_df, breast_cancer_snomed_df, id_col = 'PATIENT_IDENTIFIER', enc_col = 'ENCOUNTER_KEY', int_cols=['DIAGNOSIS_CODE'])

crc_df = unique_patients(crc_icd_df, crc_snomed_df, id_col = 'PATIENT_IDENTIFIER', enc_col = 'ENCOUNTER_KEY', int_cols=['DIAGNOSIS_CODE'])

ec_df = unique_patients(ec_icd_df, ec_snomed_df, id_col = 'PATIENT_IDENTIFIER', enc_col = 'ENCOUNTER_KEY', int_cols=['DIAGNOSIS_CODE'])

ov_df = unique_patients(ov_icd_df, ov_snomed_df, id_col = 'PATIENT_IDENTIFIER', enc_col = 'ENCOUNTER_KEY', int_cols=['DIAGNOSIS_CODE'])

pc_df = unique_patients(pc_icd_df, pc_snomed_df, id_col = 'PATIENT_IDENTIFIER', enc_col = 'ENCOUNTER_KEY', int_cols=['DIAGNOSIS_CODE'])

rc_df = unique_patients(rc_icd_df, rc_snomed_df, id_col = 'PATIENT_IDENTIFIER', enc_col = 'ENCOUNTER_KEY', int_cols=['DIAGNOSIS_CODE'])

pros_df = unique_patients(prostate_cancer_icd_df, prostate_cancer_snomed_df, id_col = 'PATIENT_IDENTIFIER', enc_col = 'ENCOUNTER_KEY', int_cols=['DIAGNOSIS_CODE'])

kc_df = unique_patients(kidney_cancer_icd_df, kidney_cancer_snomed_df, id_col = 'PATIENTPATIENT_IDENTIFIER_KEY', enc_col = 'ENCOUNTER_KEY', int_cols=['DIAGNOSIS_CODE'])

polyp_df = unique_patients(polyp_icd_df, polyp_snomed_df, id_col = 'PATIENT_IDENTIFIER', enc_col = 'ENCOUNTER_KEY', int_cols=['DIAGNOSIS_CODE'])

thyroid_df = unique_patients(thyroid_icd_df, thyroid_snomed_df, id_col = 'PATIENT_IDENTIFIER', enc_col = 'ENCOUNTER_KEY', int_cols=['DIAGNOSIS_CODE'])

diffuse_df = unique_patients(diffuse_icd_df, diffuse_snomed_df, id_col = 'PATIENT_IDENTIFIER', enc_col = 'ENCOUNTER_KEY', int_cols=['DIAGNOSIS_CODE'])

intestinal_df = unique_patients(intestinal_icd_df, intestinal_snomed_df, id_col = 'PATIENT_IDENTIFIER', enc_col = 'ENCOUNTER_KEY', int_cols=['DIAGNOSIS_CODE'])

melanoma_df = unique_patients(melanoma_icd_df, melanoma_snomed_df, id_col = 'PATIENT_IDENTIFIER', enc_col = 'ENCOUNTER_KEY', int_cols=['DIAGNOSIS_CODE'])

astrocytoma_df = unique_patients(astrocytoma_icd_df, astrocytoma_snomed_df, id_col = 'PATIENT_IDENTIFIER', enc_col = 'ENCOUNTER_KEY', int_cols=['DIAGNOSIS_CODE'])



**Assigning Diagnosis column for easier filtering**

In [ ]:
#CONVERT ALL DATE COLUMN TO DATETIME

bc_df['Diagnosis'] = 'Breast Cancer'

crc_df['Diagnosis'] = 'Colorectal Cancer'

ec_df['Diagnosis'] = 'Endometrial Cancer'

ov_df['Diagnosis'] = 'Ovarian Cancer'

pc_df['Diagnosis'] = 'Pancreatic Cancer'

rc_df['Diagnosis'] = 'Renal Cancer'

pros_df['Diagnosis'] = 'Prostate Cancer'

kc_df['Diagnosis'] = 'Kidney Cancer'

paraganglioma_df['Diagnosis'] = 'Paraganglioma'

polyp_df['Diagnosis'] = 'Polyp'

carney_df['Diagnosis'] = 'Carney Complex'

diffuse_df['Diagnosis'] = 'Diffuse Gastric Cancer'

melanoma_df['Diagnosis'] = 'Melanoma Cancer'

astrocytoma_df['Diagnosis'] = 'Astrocytoma'

In [ ]:
#Extract year into its own column

bc_df['ENCOUNTER_DATE']  = pd.to_datetime(bc_df['ENCOUNTER_DATE'])

crc_df['ENCOUNTER_DATE']  = pd.to_datetime(crc_df['ENCOUNTER_DATE'])

ec_df['ENCOUNTER_DATE']  = pd.to_datetime(ec_df['ENCOUNTER_DATE'])

ov_df['ENCOUNTER_DATE']  = pd.to_datetime(ov_df['ENCOUNTER_DATE'])

pc_df['ENCOUNTER_DATE']  = pd.to_datetime(pc_df['ENCOUNTER_DATE'])

rc_df['ENCOUNTER_DATE']  = pd.to_datetime(rc_df['ENCOUNTER_DATE'])

pros_df['ENCOUNTER_DATE']  = pd.to_datetime(pros_df['ENCOUNTER_DATE'])

kc_df['ENCOUNTER_DATE']  = pd.to_datetime(kc_df['ENCOUNTER_DATE'])

paraganglioma_df['ENCOUNTER_DATE']  = pd.to_datetime(paraganglioma_df['ENCOUNTER_DATE'])

polyp_df['ENCOUNTER_DATE']  = pd.to_datetime(polyp_df['ENCOUNTER_DATE'])

carney_df['ENCOUNTER_DATE']  = pd.to_datetime(carney_df['ENCOUNTER_DATE'])

diffuse_df['ENCOUNTER_DATE']  = pd.to_datetime(diffuse_df['ENCOUNTER_DATE'])

melanoma_df['ENCOUNTER_DATE']  = pd.to_datetime(melanoma_df['ENCOUNTER_DATE'])

astrocytoma_df['ENCOUNTER_DATE']  = pd.to_datetime(astrocytoma_df['ENCOUNTER_DATE'])

In [ ]:
bc_df['year'] = bc_df['ENCOUNTER_DATE'].dt.year
crc_df['year'] = crc_df['ENCOUNTER_DATE'].dt.year
ec_df['year'] = ec_df['ENCOUNTER_DATE'].dt.year
ov_df['year'] = ov_df['ENCOUNTER_DATE'].dt.year
pc_df['year'] = pc_df['ENCOUNTER_DATE'].dt.year
rc_df['year'] = rc_df['ENCOUNTER_DATE'].dt.year

pros_df['year'] = pros_df['ENCOUNTER_DATE'].dt.year
kc_df['year'] = kc_df['ENCOUNTER_DATE'].dt.year
paraganglioma_df['year'] = paraganglioma_df['ENCOUNTER_DATE'].dt.year
polyp_df['year'] = polyp_df['ENCOUNTER_DATE'].dt.year
carney_df['year'] = carney_df['ENCOUNTER_DATE'].dt.year

diffuse_df['year'] = diffuse_df['ENCOUNTER_DATE'].dt.year
melanoma_df['year'] = melanoma_df['ENCOUNTER_DATE'].dt.year
astrocytoma_df['year'] = astrocytoma_df['ENCOUNTER_DATE'].dt.year

In [ ]:
#Function to count yearly patients
def unique_patients(df):
    return (
        df.drop_duplicates(subset=['PATIENT_IDENTIFIER', 'year']).groupby('year')['PATIENT_IDENTIFIER'].nunique().reindex(range(2017,2022), fill_value=0)
    )

# Multi Cancer Criteria Filtering 

In [ ]:
#function to find overlapping patients in the different cancer dataframes

def overlap_cancer(df1, df2, id_col='PATIENT_IDENTIFIER', year_col='year'):
    
    id1 = set(df1[id_col].dropna().unique())
    id2 = set(df2[id_col].dropna().unique())
    
    overlap_ids = id1.intersection(id2)
    
    print(f"{len(overlap_ids)} unique patient(s) found in both DataFrames.")
    
    overlap_df = pd.DataFrame({id_col: list(overlap_ids)})
    
    combined = pd.concat([df1, df2], ignore_index=True)
    combined = combined[combined[id_col].isin(overlap_ids)]
    
    combined_unique = combined.drop_duplicates(subset=[id_col, year_col])
    
    yearly_counts = (
        combined_unique.groupby(year_col)[id_col].nunique().reindex(range(2017,2022), fill_value=0)
    )
    
    for year, count in yearly_counts.items():
        print(f"Year {year}: {count}")
    
    return overlap_df, yearly_counts

**Filter 1: Prostate Cancer and also diagnosed with male breast cancer**

In [ ]:
m_bc = bc_df[bc_df['GENDER'] == 'Male']
m_bc_cnt = m_bc['PATIENT_IDENTIFIER'].nunique()
print(f"There are {m_bc_cnt} Male Breast Cancer Patients")

In [ ]:
pc_mc_yearly, pc_mc = overlap_cancer(pros_df, m_bc, id_col='PATIENT_IDENTIFIER', year_col='year')

In [ ]:
pc_mc_yearly

**Filter 2: Kidney cancer and paraganglioma or phaeochromocytoma**

In [ ]:
kc_p = overlap_cancer(paraganglioma_df, kc_df, id_col='PATIENT_IDENTIFIER')

**Filter 3: Kidney Cancer and Polyp**

In [ ]:
kc_polyp_yearly, kc_polyp = overlap_cancer(polyp_df, kc_df, id_col='PATIENT_IDENTIFIER')

**Filter 4: Kidney Cancer and Thyroid Cancer**

In [ ]:
kc_thy_yearly, kc_thy = overlap_cancer(thyroid_df, kc_df, id_col='PATIENT_IDENTIFIER')

**Filter 5: Kidney Cancer and Endometiral Cancer**

In [ ]:
kc_ec_yearly, kc_ec = overlap_cancer(ec_df, kc_df, id_col='PATIENT_IDENTIFIER')

**Filter 6: Thyroid cancer and carney complex criterion**

In [ ]:
tc_cc_yearly, tc_cc = overlap_cancer(thyroid_df, carney_df, id_col='PATIENT_IDENTIFIER')

**Filter 7: Thyroid Cancer and Polyp**

In [ ]:
tc_p_yearly, tc_p = overlap_cancer(thyroid_df, polyp_df, id_col='PATIENT_IDENTIFIER')

**Filter 8: Thyroid Cancer and Endometrial Cancer**

In [ ]:
tc_ec_yearly, tc_ec = overlap_cancer(thyroid_df, ec_df, id_col='PATIENT_IDENTIFIER')

**Filter 9: Diffuse gastric cancer and lobular bc in same person both diagnosed age < 70**

In [ ]:
#Diffuse gastric cancer age < 70
ag_dgc = diffuse_df[diffuse_df['AGE'] < 70]

#Breast cancer age < 70
ag_bc = breast_cancer_icd_df[breast_cancer_icd_df['AGE'] < 70]

#overlap
dgc_bc_yearly, dgc_bc = overlap_cancer(ag_dgc, ag_bc, id_col='PATIENT_IDENTIFIER')

**Filter 10: Gastric Cancer and intestinal polyps**

In [ ]:
gc_ip_yearly, gc_ip = overlap_cancer(intestinal_df, diffuse_df, id_col='PATIENT_IDENTIFIER')

**Filter 11: Melanoma and pancreatic cancer**

In [ ]:
mc_pc_yearly, mc_pc = overlap_cancer(melanoma_df, pc_df, id_col='PATIENT_IDENTIFIER')

**Filter 12: melanoma and astrocytoma**

In [ ]:
mc_a_yearly, mc_a = overlap_cancer(astrocytoma_df, melanoma_df, id_col = 'PATIENT_IDENTIFIER')

In [ ]:
# ALL FLAGGED PATIENT KEY
multi_flagged_df = pd.concat([mc_a_yearly, pc_mc_yearly, mc_pc_yearly, gc_ip_yearly, dgc_bc_yearly, tc_ec_yearly, tc_p_yearly, tc_cc_yearly, kc_ec_yearly, kc_thy_yearly, kc_polyp_yearly], ignore_index=True)

In [ ]:
multi_flagged_df['PATIENT_IDENTIFIER'] = multi_flagged_df['PATIENT_IDENTIFIER'].astype('Int64')

In [ ]:
multi_flagged_df.to_csv('multi_flagged_df.csv')